# Locality Sensitive hashing in Collaborative item-item filtering (inspired by week5 : SimilarItems)

Explain:
- LSH vs KNN, and why LSH is more efficient
- Present a scheme of how LSH algorithm operates
- Why we use Jaccard distance here (and how we make the decision for binary outcome, "likes":1/"dislikes":0 )

Imports

In [1]:
import numpy as np
import pandas as pd
import time
from datasketch import MinHash, MinHashLSHForest

Read in the data:
    
    userID \tab songID \tab play_count

In [2]:
u_matrix = pd.read_csv('triplets_50000.txt', sep='\t', names = ['userID','songID', 'play_count'])

Pivot to tranform the data from long to wide:

In [3]:
u_matrix = u_matrix.pivot(index="userID", columns="songID", values="play_count")

In [4]:
u_matrix

songID,SOAAAGQ12A8C1420C8,SOAACPJ12A81C21360,SOAACTC12AB0186A20,SOAADCB12A81C22AFA,SOAAEJI12AB0188AB5,SOAAEKX12A6D4F7E4E,SOAAFAC12A67ADF7EB,SOAAFYH12A8C13717A,SOAAIJG12AAA15D821,SOAAIWE12A8AE4706B,...,SOZZVFP12A8C140F14,SOZZVMW12AB0183B52,SOZZVNT12AF729EBC9,SOZZVWB12AB0189C30,SOZZWZV12A67AE140F,SOZZXAO12A58A7D379,SOZZYAO12A6701FF36,SOZZYDA12AB01824FB,SOZZYMH12AB0180A51,SOZZZFB12A8AE45CDC
userID,,,,,,,,,,,,,,,,,,,,,
0007c0e74728ca9ef0fe4eb7f75732e8026a278b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0039bd8483d578997718cdc0bf6c7c88b679f488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00498f4bab2bfeb17680113c7d9525ad5b0ad401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00a443baf550f4bbdd974ba73720abf2759166f3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0152fcbd02b172a874c75a57a913f0f0109ba272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe979a7b199de3ee8a78486c10e5ed13587fc359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fed37c4c49c9f217b3371c2f2c0e7541656e55cf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ff18ea9a13583f7f7aaa83719e0b22ce5618e9cf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# u_m = u_matrix.fillna(0) # Replace the NaN with 0s.

In [6]:
total_count_list = u_matrix.sum(axis=1, skipna=True)

In [7]:
print("The greatest listener's total play count is", np.max(total_count_list))
print("The lowest total play count of a user is", np.min(total_count_list))

The greatest listener's total play count is 2478.0
The lowest total play count of a user is 10.0


So it's a good idea, before we start, to normalize for "big" and "low" listeners:

In [5]:
u_normalized = u_matrix.sub(u_matrix.mean(axis=1, skipna=True), axis=0) # substract from each cell the row mean

In [9]:
u_normalized.shape

(1010, 27898)

Lets use the rule:

    if the norm play count is < 0 == the user didn't like the song
    if the norm play count is > 0 == the user likes the song

In [10]:
# u_binary = np.where(u_normalized[u_normalized.columns] < 0, 0, 1)

In [6]:
df = u_normalized.copy()

In [7]:
df[df >= 0] = 1
df[df < 0] = 0

In [15]:
df.shape

(1010, 27898)

In [8]:
df = df.T
df

userID,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,03c90bfd09151973863c4cadd5a749cd7982abc0,...,fca15b7964a099b2860dfdb158a2430fb10c4384,fd13b9d49c54e00ff413fe3c095ba581c7fc611e,fd585aef5c32d3943bd6e7f9f39aa216ba659fd0,fdb815231ee1d66f383b80d279bd58769dfe59ff,fe76c9d535c5834e4a9b91c13e29be6460cb79c4,fe979a7b199de3ee8a78486c10e5ed13587fc359,fed37c4c49c9f217b3371c2f2c0e7541656e55cf,ff18ea9a13583f7f7aaa83719e0b22ce5618e9cf,ff4322e94814d3c7895d07e6f94139b092862611,ffadf9297a99945c0513cd87939d91d8b602936b
songID,,,,,,,,,,,,,,,,,,,,,
SOAAAGQ12A8C1420C8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAACPJ12A81C21360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAACTC12AB0186A20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAADCB12A81C22AFA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAAEJI12AB0188AB5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOZZXAO12A58A7D379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOZZYAO12A6701FF36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOZZYDA12AB01824FB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Transform the data for the LSH algo
start_time = time.time()
cols = df.columns.to_numpy() # the users 
vectors_list = [cols[x].tolist() for x in df.eq(1).to_numpy()] # each vector is a song, contains users that liked that song
print('It took %s seconds.' %(time.time()-start_time))

It took 0.1858839988708496 seconds.


In [ ]:
# sanity check
len(vectors_list) # should be equal to the 27898 columns (songs)

In [1]:
df_new = pd.DataFrame(vectors_list, index = df.index)
# df_new.shape # (27898, 49)
# drop rows with none
df_new.mask(df_new.eq('None')).dropna() 
df_new.shape

In [22]:
df_new_reduced = df_new.mask(df_new.eq('None')).dropna(how = 'all') 

In [23]:
df_new_reduced.index

Index(['SOAAAGQ12A8C1420C8', 'SOAACPJ12A81C21360', 'SOAACTC12AB0186A20',
       'SOAADCB12A81C22AFA', 'SOAAEJI12AB0188AB5', 'SOAAEKX12A6D4F7E4E',
       'SOAAFAC12A67ADF7EB', 'SOAAFYH12A8C13717A', 'SOAAIJG12AAA15D821',
       'SOAAIWE12A8AE4706B',
       ...
       'SOZZVFP12A8C140F14', 'SOZZVMW12AB0183B52', 'SOZZVNT12AF729EBC9',
       'SOZZVWB12AB0189C30', 'SOZZWZV12A67AE140F', 'SOZZXAO12A58A7D379',
       'SOZZYAO12A6701FF36', 'SOZZYDA12AB01824FB', 'SOZZYMH12AB0180A51',
       'SOZZZFB12A8AE45CDC'],
      dtype='object', name='songID', length=27898)

In [24]:
df_new_reduced['users'] = df_new_reduced[df_new_reduced.columns].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df_new_reduced

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,users
songID,,,,,,,,,,,,,,,,,,,,,
SOAAAGQ12A8C1420C8,3ff7a31452eeabd7a4e07f0d243c674e3d0adf46,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3ff7a31452eeabd7a4e07f0d243c674e3d0adf46
SOAACPJ12A81C21360,fb2f2c0e39e233622c300c232a6287738007e34a,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,fb2f2c0e39e233622c300c232a6287738007e34a
SOAACTC12AB0186A20,cd7baa603618c40b9290bd05112061afefdde8ef,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,cd7baa603618c40b9290bd05112061afefdde8ef
SOAADCB12A81C22AFA,18dc95f354220b343e98ebbb7c8564291284ed9f,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,18dc95f354220b343e98ebbb7c8564291284ed9f
SOAAEJI12AB0188AB5,28b232e7ecb32c47c05b795a017786d4be96ef7e,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,28b232e7ecb32c47c05b795a017786d4be96ef7e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOZZXAO12A58A7D379,4208d4ac45e7caab7167a4ea6d34e759a6b9a1fc,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4208d4ac45e7caab7167a4ea6d34e759a6b9a1fc
SOZZYAO12A6701FF36,5a905f000fc1ff3df7ca807d57edb608863db05d,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,5a905f000fc1ff3df7ca807d57edb608863db05d
SOZZYDA12AB01824FB,2c218a60b3d777e9e12d56c2e065a9644b5e5f41,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2c218a60b3d777e9e12d56c2e065a9644b5e5f41


In [25]:
data = df_new_reduced['users'].copy()

In [26]:
data = pd.DataFrame(data)

In [27]:
data

,users
songID,
SOAAAGQ12A8C1420C8,3ff7a31452eeabd7a4e07f0d243c674e3d0adf46
SOAACPJ12A81C21360,fb2f2c0e39e233622c300c232a6287738007e34a
SOAACTC12AB0186A20,cd7baa603618c40b9290bd05112061afefdde8ef
SOAADCB12A81C22AFA,18dc95f354220b343e98ebbb7c8564291284ed9f
SOAAEJI12AB0188AB5,28b232e7ecb32c47c05b795a017786d4be96ef7e
...,...
SOZZXAO12A58A7D379,4208d4ac45e7caab7167a4ea6d34e759a6b9a1fc
SOZZYAO12A6701FF36,5a905f000fc1ff3df7ca807d57edb608863db05d
SOZZYDA12AB01824FB,2c218a60b3d777e9e12d56c2e065a9644b5e5f41


Choose parameters:

In [28]:
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

Create MiniHash forest

In [29]:
#Preprocess will split a string of text into individual tokens/shingles based on ",".
def preprocess(text):
    tokens = text.split()
    return tokens


In [30]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for users in data['users']:
        tokens = preprocess(users) # list of users
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest


In [31]:
forest = get_forest(data, permutations)

It took 52.37690019607544 seconds to build forest.


In [32]:
def predict(song_profile, database, perms, num_results, forest): # song_profile in list form
    start_time = time.time()
    
    m = MinHash(num_perm=perms)
    for users in song_profile:
        m.update(users.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['users']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result


In [37]:
num_recommendations = 10
song_profile = [ '18dc95f354220b343e98ebbb7c8564291284ed9f', '5a905f000fc1ff3df7ca807d57edb608863db05d'] # i made this random profile of an imaginary user manually, make it work with songID
result = predict(song_profile, data, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)


It took 0.008995771408081055 seconds to query forest.

 Top Recommendation(s) is(are) 
 songID
SOBVPFU12A6D4FB176    18dc95f354220b343e98ebbb7c8564291284ed9f
SOAADCB12A81C22AFA    18dc95f354220b343e98ebbb7c8564291284ed9f
SOVESPW12AB0179EC6    18dc95f354220b343e98ebbb7c8564291284ed9f
SODQKXS12A8C135DF7    5a905f000fc1ff3df7ca807d57edb608863db05d
SONVRTN12A8C1385C2    5a905f000fc1ff3df7ca807d57edb608863db05d
                                        ...                   
SOGVWGI12A8C13B9D1    5a905f000fc1ff3df7ca807d57edb608863db05d
SONJJGP12A67020D47    18dc95f354220b343e98ebbb7c8564291284ed9f
SOOHCOB12A6D4F995E    18dc95f354220b343e98ebbb7c8564291284ed9f
SORGOJJ12A6D4F8597    5a905f000fc1ff3df7ca807d57edb608863db05d
SOAXTWB12AF72A1545    18dc95f354220b343e98ebbb7c8564291284ed9f
Name: users, Length: 152, dtype: object
